# Agglomerative Clustering

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Import to dataframe
df_features = pd.read_csv('processed_hotel_bookings.csv')

In [3]:
#The dataset is quite large, and I am getting memory errors on this particular algorithm.
#I will split it randomnly.
df_features = df_features.sample(frac=0.05, random_state=763)

#We need to reset the index so the the algorithms below can loop through the dataframe
df_features = df_features.reset_index()

### Data Prep with t-SNE dimensionality reduction

Elaboration of my dimensionality reduction configuration decision process is given in the Final Dimension Reduction Configuration notebook.

In [4]:
#Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

In [5]:
# Loading the data
X = df_features
y = df_features.is_canceled

# Standarizing the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

X_norm = normalize(X_std)

In [6]:
from sklearn.manifold import TSNE
import time

In [7]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=100, n_iter=300)
tsne_results = tsne.fit_transform(X_std)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 301 nearest neighbors...
[t-SNE] Indexed 1978 samples in 0.017s...
[t-SNE] Computed neighbors for 1978 samples in 0.289s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1978
[t-SNE] Computed conditional probabilities for sample 1978 / 1978
[t-SNE] Mean sigma: 1.125884
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.459862
[t-SNE] KL divergence after 300 iterations: 0.497429
t-SNE done! Time elapsed: 8.61219596862793 seconds


In the figure below, red shows the bookings that were not canceled, and blue shows the bookings that were.

## Agglomerative Clustering

In [8]:
#Libraries
from sklearn import datasets
from sklearn import metrics

In [9]:
#Libraries
from sklearn.cluster import AgglomerativeClustering

### The Best Performer
The best ARI and Silhouetter scores were produced by the 3 cluster algorithm with single Linkage and cosine affinity

In [10]:
# Defining the agglomerative clustering
agg_cluster = AgglomerativeClustering(linkage='complete', 
                                      affinity='l1',
                                      n_clusters=3)

# Fit model
A_clusters = agg_cluster.fit_predict(tsne_results)

In [11]:
plt.figure(figsize=(10,5))
colours = 'rbg'
for i in range(tsne_results.shape[0]):
    plt.text(tsne_results[i, 0], tsne_results[i, 1], str(A_clusters[i]),
             color=colours[y[i]],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

Red is not cancelled, blue is cancelled.

In [12]:
A_clusters = np.array([0 if x == 1 else 1 if x==0 else 2 for x in A_clusters])

pd.crosstab(y, A_clusters)

col_0,0,1,2
is_canceled,,,
0,112,883,471
1,510,2,0


## Finding the best performer

### 2 Clusters

In [13]:
def agg_ari_silh(linkage, affinity, x, y):
  pred = AgglomerativeClustering(linkage=linkage, 
                                 affinity=affinity,
                                 n_clusters=2).fit_predict(tsne_results)
  print("Adjusted Rand Index of the Agglomerative Clustering solution ({}/{}) with {} clusters: {}"
      .format(linkage, affinity, 2, metrics.adjusted_rand_score(y, pred)))
  print("The silhoutte score of the Agglomerative Clustering solution ({}/{}) with {} clusters: {}"
      .format(linkage, affinity, 2, metrics.silhouette_score(x, pred, metric='euclidean')))
  print('='*90)

In [14]:
# check agglomerative clustering 
agg_ari_silh('ward', 'euclidean', tsne_results, y)
linkage = ['complete', 'average', 'single']
affinity = ['l1', 'l2', 'manhattan', 'cosine', 'euclidean']
print('\n')
for i in linkage:
  for j in affinity:
    print(i.upper()+'/'+j.upper())
    agg_ari_silh(i, j, X_std, y)
    print('\n')

Adjusted Rand Index of the Agglomerative Clustering solution (ward/euclidean) with 2 clusters: 0.164396718075845
The silhoutte score of the Agglomerative Clustering solution (ward/euclidean) with 2 clusters: 0.3830720782279968


COMPLETE/L1
Adjusted Rand Index of the Agglomerative Clustering solution (complete/l1) with 2 clusters: 0.164396718075845
The silhoutte score of the Agglomerative Clustering solution (complete/l1) with 2 clusters: 0.12818012307024634


COMPLETE/L2
Adjusted Rand Index of the Agglomerative Clustering solution (complete/l2) with 2 clusters: 0.16597445113440265
The silhoutte score of the Agglomerative Clustering solution (complete/l2) with 2 clusters: 0.12854824168295687


COMPLETE/MANHATTAN
Adjusted Rand Index of the Agglomerative Clustering solution (complete/manhattan) with 2 clusters: 0.164396718075845
The silhoutte score of the Agglomerative Clustering solution (complete/manhattan) with 2 clusters: 0.12818012307024634


COMPLETE/COSINE
Adjusted Rand Index of t

### 3 clusters

In [15]:
def agg_ari_silh(linkage, affinity, x, y):
  pred = AgglomerativeClustering(linkage=linkage, 
                                 affinity=affinity,
                                 n_clusters=3).fit_predict(tsne_results)
  print("Adjusted Rand Index of the Agglomerative Clustering solution ({}/{}) with {} clusters: {}"
      .format(linkage, affinity, 3, metrics.adjusted_rand_score(y, pred)))
  print("The silhoutte score of the Agglomerative Clustering solution ({}/{}) with {} clusters: {}"
      .format(linkage, affinity, 3, metrics.silhouette_score(x, pred, metric='euclidean')))
  print('='*90)

In [16]:
# check agglomerative clustering 
agg_ari_silh('ward', 'euclidean', tsne_results, y)
linkage = ['complete', 'average', 'single']
affinity = ['l1', 'l2', 'manhattan', 'cosine', 'euclidean']
print('\n')
for i in linkage:
  for j in affinity:
    print(i.upper()+'/'+j.upper())
    agg_ari_silh(i, j, X_std, y)
    print('\n')

Adjusted Rand Index of the Agglomerative Clustering solution (ward/euclidean) with 3 clusters: 0.5002301289568604
The silhoutte score of the Agglomerative Clustering solution (ward/euclidean) with 3 clusters: 0.4386309087276459


COMPLETE/L1
Adjusted Rand Index of the Agglomerative Clustering solution (complete/l1) with 3 clusters: 0.39857324122702986
The silhoutte score of the Agglomerative Clustering solution (complete/l1) with 3 clusters: 0.1579697411888258


COMPLETE/L2
Adjusted Rand Index of the Agglomerative Clustering solution (complete/l2) with 3 clusters: 0.4152756140271437
The silhoutte score of the Agglomerative Clustering solution (complete/l2) with 3 clusters: 0.20133152198675458


COMPLETE/MANHATTAN
Adjusted Rand Index of the Agglomerative Clustering solution (complete/manhattan) with 3 clusters: 0.39857324122702986
The silhoutte score of the Agglomerative Clustering solution (complete/manhattan) with 3 clusters: 0.1579697411888258


COMPLETE/COSINE
Adjusted Rand Index of

### 4 clusters

In [17]:
def agg_ari_silh(linkage, affinity, x, y):
  pred = AgglomerativeClustering(linkage=linkage, 
                                 affinity=affinity,
                                 n_clusters=4).fit_predict(tsne_results)
  print("Adjusted Rand Index of the Agglomerative Clustering solution ({}/{}) with {} clusters: {}"
      .format(linkage, affinity, 4, metrics.adjusted_rand_score(y, pred)))
  print("The silhoutte score of the Agglomerative Clustering solution ({}/{}) with {} clusters: {}"
      .format(linkage, affinity, 4, metrics.silhouette_score(x, pred, metric='euclidean')))
  print('='*90)

In [18]:
# check agglomerative clustering 
agg_ari_silh('ward', 'euclidean', tsne_results, y)
linkage = ['complete', 'average', 'single']
affinity = ['l1', 'l2', 'manhattan', 'cosine', 'euclidean']
print('\n')
for i in linkage:
  for j in affinity:
    print(i.upper()+'/'+j.upper())
    agg_ari_silh(i, j, X_std, y)
    print('\n')

Adjusted Rand Index of the Agglomerative Clustering solution (ward/euclidean) with 4 clusters: 0.362586931906729
The silhoutte score of the Agglomerative Clustering solution (ward/euclidean) with 4 clusters: 0.5021250247955322


COMPLETE/L1
Adjusted Rand Index of the Agglomerative Clustering solution (complete/l1) with 4 clusters: 0.26450053985805433
The silhoutte score of the Agglomerative Clustering solution (complete/l1) with 4 clusters: 0.19512465528239642


COMPLETE/L2
Adjusted Rand Index of the Agglomerative Clustering solution (complete/l2) with 4 clusters: 0.2797418934407104
The silhoutte score of the Agglomerative Clustering solution (complete/l2) with 4 clusters: 0.23351904077361368


COMPLETE/MANHATTAN
Adjusted Rand Index of the Agglomerative Clustering solution (complete/manhattan) with 4 clusters: 0.26450053985805433
The silhoutte score of the Agglomerative Clustering solution (complete/manhattan) with 4 clusters: 0.19512465528239642


COMPLETE/COSINE
Adjusted Rand Index o

### 5 clusters

In [19]:
def agg_ari_silh(linkage, affinity, x, y):
  pred = AgglomerativeClustering(linkage=linkage, 
                                 affinity=affinity,
                                 n_clusters=5).fit_predict(tsne_results)
  print("Adjusted Rand Index of the Agglomerative Clustering solution ({}/{}) with {} clusters: {}"
      .format(linkage, affinity, 5, metrics.adjusted_rand_score(y, pred)))
  print("The silhoutte score of the Agglomerative Clustering solution ({}/{}) with {} clusters: {}"
      .format(linkage, affinity, 5, metrics.silhouette_score(x, pred, metric='euclidean')))
  print('='*90)

In [20]:
# check agglomerative clustering 
agg_ari_silh('ward', 'euclidean', tsne_results, y)
linkage = ['complete', 'average', 'single']
affinity = ['l1', 'l2', 'manhattan', 'cosine', 'euclidean']
print('\n')
for i in linkage:
  for j in affinity:
    print(i.upper()+'/'+j.upper())
    agg_ari_silh(i, j, X_std, y)
    print('\n')

Adjusted Rand Index of the Agglomerative Clustering solution (ward/euclidean) with 5 clusters: 0.31439421059922096
The silhoutte score of the Agglomerative Clustering solution (ward/euclidean) with 5 clusters: 0.4731711447238922


COMPLETE/L1
Adjusted Rand Index of the Agglomerative Clustering solution (complete/l1) with 5 clusters: 0.2419637113212314
The silhoutte score of the Agglomerative Clustering solution (complete/l1) with 5 clusters: 0.19643327243692685


COMPLETE/L2
Adjusted Rand Index of the Agglomerative Clustering solution (complete/l2) with 5 clusters: 0.30576376318177056
The silhoutte score of the Agglomerative Clustering solution (complete/l2) with 5 clusters: 0.207183590683836


COMPLETE/MANHATTAN
Adjusted Rand Index of the Agglomerative Clustering solution (complete/manhattan) with 5 clusters: 0.2419637113212314
The silhoutte score of the Agglomerative Clustering solution (complete/manhattan) with 5 clusters: 0.19643327243692685


COMPLETE/COSINE
Adjusted Rand Index of